In [ ]:
import tensorflow as tf
import os
import numpy as np
import csv
import lpips
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications.resnet import preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.resnet import decode_predictions

In [ ]:
# Configure parameters
EPSILONS = [0, 2, 4, 8, 16, 32]  # Different perturbation strengths
IMAGE_DIR = "/kaggle/input/tiny-imagenet/tiny-imagenet-200/test/images"
RESULTS_FILE = "attack_results.csv"
NUM_IMAGES = 20  # Adjust based on your needs

In [ ]:
# Initialize ResNet101
model = ResNet101(weights='imagenet')
input_size = (224, 224)

In [ ]:
# Define reverse preprocessing for SSIM calculation
def reverse_preprocess(adv_image):
    mean = [103.939, 116.779, 123.68]
    reversed_img = adv_image.copy()
    reversed_img[..., 0] += mean[0]
    reversed_img[..., 1] += mean[1]
    reversed_img[..., 2] += mean[2]
    reversed_img = reversed_img[..., ::-1]  # BGR to RGB
    return np.clip(reversed_img, 0, 255).astype('uint8')

# FGSM attack function
def fgsm_attack(model, input_image, epsilon):
    input_tensor = tf.convert_to_tensor(input_image)
    with tf.GradientTape() as tape:
        tape.watch(input_tensor)
        prediction = model(input_tensor)
        loss = tf.keras.losses.categorical_crossentropy(prediction, prediction)
    gradient = tape.gradient(loss, input_tensor)
    perturbation = epsilon * tf.sign(gradient)
    return input_tensor + perturbation

In [ ]:
# Load LPIPS model (using AlexNet, but can also use 'vgg' or 'squeeze')
lpips_model = tf_lpips.LPIPS('net-lin', 'alex')

In [ ]:
# Compute LPIPS similarity
# lpips_value = lpips_model(img1_torch, img2_torch)

In [ ]:
RESULTS_FILE = "FGSM attacks.csv"
results = []
image_files = [f for f in os.listdir(IMAGE_DIR) if f.endswith(".JPEG")][:NUM_IMAGES]

for epsilon in EPSILONS:
    for img_file in image_files:
        try:
            # Load and preprocess
            img_path = os.path.join(IMAGE_DIR, img_file)
            original, processed = preprocess_image(img_path)

            # Generate adversarial
            adversarial = fgsm_attack(model, processed, epsilon).numpy()
            adversarial = np.clip(adversarial, -100.0, 150.0)

            # Calculate metrics
            adv_reversed = reverse_preprocess(adversarial[0])
            ssim = tf.image.ssim(
                tf.convert_to_tensor([original], tf.uint8),
                tf.convert_to_tensor([adv_reversed], tf.uint8),
                max_val=255
            ).numpy()[0]

            # Normalize images for LPIPS [-1, 1]
            original_lpips = (original / 127.5) - 1.0
            adversarial_lpips = (adv_reversed / 127.5) - 1.0

            # Convert to TensorFlow tensors and add batch dimension
            original_tf = tf.convert_to_tensor(original_lpips, dtype=tf.float32)
            adversarial_tf = tf.convert_to_tensor(adversarial_lpips, dtype=tf.float32)
            original_tf = tf.expand_dims(original_tf, axis=0)
            adversarial_tf = tf.expand_dims(adversarial_tf, axis=0)

            # Compute LPIPS similarity
            lpips_value = lpips_model(original_tf, adversarial_tf).numpy()[0]

            # Get predictions with confidence
            clean_preds = model.predict(processed)
            adv_preds = model.predict(adversarial)

            # Decode top predictions
            clean_top = decode_predictions(clean_preds, top=1)[0][0]
            adv_top = decode_predictions(adv_preds, top=1)[0][0]

            results.append({
                'Image': img_file,
                'Epsilon': epsilon,
                'SSIM': ssim,
                'LPIPS': lpips,
                'CleanLabel': clean_top[1],  # Class label
                'CleanConfidence': float(clean_top[2]),  # Confidence score
                'AdvLabel': adv_top[1],
                'AdvConfidence': float(adv_top[2]),
                'Success': int(clean_top[1] != adv_top[1])
            })

        except Exception as e:
            print(f"Error with {img_file}: {str(e)}")

In [ ]:
# Save to CSV with specified filename
with open(RESULTS_FILE, 'w', newline='') as csvfile:
    fieldnames = [
        'Image', 'Epsilon', 'SSIM', 'LPIPS',
        'CleanLabel', 'CleanConfidence',
        'AdvLabel', 'AdvConfidence',
        'Success'
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(results)

print(f"FGSM attack results saved to {RESULTS_FILE}")

In [ ]:
# -*- coding: utf-8 -*-
"""FGSM_Attack_Corrected"""

import os
import numpy as np
import csv
import tensorflow as tf
import torch
import lpips
from tensorflow.keras.applications import ResNet101
from tensorflow.keras.applications.resnet50 import preprocess_input, decode_predictions
from tensorflow.keras.preprocessing import image

# Configure parameters
EPSILONS = [0, 2, 4, 8, 16, 32]  # Different perturbation strengths
IMAGE_DIR = "/kaggle/input/tiny-imagenet/tiny-imagenet-200/test/images"
RESULTS_FILE = "FGSM_attack_results.csv"
NUM_IMAGES = 20  # Adjust based on your needs

# Initialize ResNet101 model
model = ResNet101(weights='imagenet')
input_size = (224, 224)

# Define image preprocessing function
def preprocess_image(img_path):
    img = image.load_img(img_path, target_size=input_size)
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    return img_array, preprocess_input(img_array)

# Reverse preprocessing for SSIM calculation
def reverse_preprocess(adv_image):
    mean = [103.939, 116.779, 123.68]
    reversed_img = adv_image.copy()
    reversed_img[..., 0] += mean[0]
    reversed_img[..., 1] += mean[1]
    reversed_img[..., 2] += mean[2]
    reversed_img = reversed_img[..., ::-1]  # Convert BGR to RGB
    return np.clip(reversed_img, 0, 255).astype('uint8')

# FGSM attack function
def fgsm_attack(model, input_image, true_label, epsilon):
    input_tensor = tf.convert_to_tensor(input_image, dtype=tf.float32)
    with tf.GradientTape() as tape:
        tape.watch(input_tensor)
        prediction = model(input_tensor)
        loss = tf.keras.losses.sparse_categorical_crossentropy(true_label, prediction)
    gradient = tape.gradient(loss, input_tensor)
    perturbation = epsilon * tf.sign(gradient)
    adversarial_example = input_tensor + perturbation
    return tf.clip_by_value(adversarial_example, 0, 255).numpy()

# Load LPIPS model
lpips_model = lpips.LPIPS(net='alex')

# Process images and generate adversarial examples
results = []
image_files = [f for f in os.listdir(IMAGE_DIR) if f.endswith(".JPEG")][:NUM_IMAGES]

for epsilon in EPSILONS:
    for img_file in image_files:
        try:
            # Load and preprocess image
            img_path = os.path.join(IMAGE_DIR, img_file)
            original, processed = preprocess_image(img_path)

            # Get true label
            clean_preds = model.predict(processed)
            true_label = np.argmax(clean_preds)

            # Generate adversarial image
            adversarial = fgsm_attack(model, processed, true_label, epsilon)

            # Reverse preprocess for SSIM
            adv_reversed = reverse_preprocess(adversarial[0])
            ssim = tf.image.ssim(
                tf.convert_to_tensor(original, dtype=tf.uint8),
                tf.convert_to_tensor(adv_reversed, dtype=tf.uint8),
                max_val=255
            ).numpy()[0]

            # Normalize images for LPIPS
            original_lpips = (original / 127.5) - 1.0
            adversarial_lpips = (adv_reversed / 127.5) - 1.0

            # Convert to PyTorch tensors
            original_torch = torch.tensor(original_lpips, dtype=torch.float32).permute(0, 3, 1, 2)
            adversarial_torch = torch.tensor(adversarial_lpips, dtype=torch.float32).permute(0, 3, 1, 2)

            # Compute LPIPS similarity
            lpips_value = lpips_model(original_torch, adversarial_torch).item()

            # Get predictions for adversarial image
            adv_preds = model.predict(adversarial)
            clean_top = decode_predictions(clean_preds, top=1)[0][0]
            adv_top = decode_predictions(adv_preds, top=1)[0][0]

            # Store results
            results.append({
                'Image': img_file,
                'Epsilon': epsilon,
                'SSIM': ssim,
                'LPIPS': lpips_value,
                'CleanLabel': clean_top[1],
                'CleanConfidence': float(clean_top[2]),
                'AdvLabel': adv_top[1],
                'AdvConfidence': float(adv_top[2]),
                'Success': int(clean_top[1] != adv_top[1])
            })

        except Exception as e:
            print(f"Error processing {img_file}: {e}")

# Save results to CSV
with open(RESULTS_FILE, 'w', newline='') as csvfile:
    fieldnames = [
        'Image', 'Epsilon', 'SSIM', 'LPIPS', 'CleanLabel', 'CleanConfidence',
        'AdvLabel', 'AdvConfidence', 'Success'
    ]
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    writer.writerows(results)

print(f"FGSM attack results saved to {RESULTS_FILE}")